In [ ]:
!gcloud config list

In [ ]:
!pip install datasets

In [1]:
from vertexai import init
from vertexai.preview.generative_models import GenerativeModel

init(project="gen-lang-client-0947040378", location="us-central1")
model = GenerativeModel("gemini-2.5-flash-preview-05-20")

In [3]:
import json
import re
import pandas as pd
from tqdm import tqdm
import glob
import time
from vertexai.preview.generative_models import GenerativeModel

def extract_choices_from_prompt(text):
    return set(re.findall(r"\b([ABC])\s*:", text.upper()))

def extract_choices_from_response(text):
    text = text.upper()
    match = re.search(r"RĂSPUNS(?:\s+CORECT)?\s*[:：]?\s*([ABC](?:[\s,]+[ABC])*)", text)
    if match:
        return set(re.findall(r"[ABC]", match.group(1)))
    return None

def evaluate_gemini(model, dataset, max_examples=None, verbose=False, csv_path=None,multimodal=False):
    results = []
    exact_matches = 0
    
    for i, ex in enumerate(tqdm(dataset, desc="Evaluare QA", unit="ex")):
        if max_examples and i >= max_examples:
            break
        try:
            prompt = None
            if multimodal:
                image_text=str(ex.get("image", "")).strip()
                prompt = (
                  image_text+"\n"
                  + ex["prompt"].strip()
                  + "\nMai întâi oferă doar litera/literele răspunsului corect sub forma:\nRăspuns: A\n"
                  + "Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.\n"
                  + "Răspuns:"
                  )
            else:
                prompt = (
                  ex["prompt"].strip()
                  + "\nMai întâi oferă doar litera/literele răspunsului corect sub forma:\nRăspuns: A\n"
                  + "Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.\n"
                  + "Răspuns:"
                  )
            gold = extract_choices_from_response(ex["response"])

            response = model.generate_content(
                prompt
            )

            text = getattr(response, "text", "").strip()
            pred = extract_choices_from_response(text)

            if verbose:
                print(f"\n📘 Prompt: {prompt}")
                print(f"✅ Gold: {gold}")
                print(f"🧠 Pred: {pred}")
                print(f"📝 Gemini raw: {text}")

            match = gold == pred if gold and pred else False
            if match:
                exact_matches += 1

            results.append({
                "index":str(i),
                "prompt": ex["prompt"].strip(),
                "golds": ", ".join(sorted(gold)) if gold else "",
                "preds": ", ".join(sorted(pred)) if pred else "",
                "gemini_raw": text
            })

        except Exception as e:
            print(f"[❌ EROARE] {e}")
            continue

    # Accuracy
    accuracy = exact_matches / len(results) if results else 0
    print(f"\n📊 Acuratețe totală (exact match): {accuracy:.2%}")

    # Export CSV
    if csv_path:
        df = pd.DataFrame(results)
        df.to_csv(csv_path, index=False, encoding="utf-8")
        print(f"📁 Rezultate salvate în: {csv_path}")

    return results, accuracy


In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
with open("multimodal.jsonl", "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]
    dataset_text_only = Dataset.from_list(dataset)

    split_dataset = dataset_text_only.train_test_split(test_size=0.2, seed=42)
    train_ds = split_dataset["train"]
    test_ds = split_dataset["test"]
    print(test_ds[1])

{'image': 'https://huggingface.co/datasets/RKThirteen/ImaginiExamenAuto/resolve/main/Cat_B_poz_375.jpg', 'prompt': 'Imaginea conține o situație rutieră. Răspunde pe baza acesteia și a întrebării de mai jos.\nÎntrebare: În situația dată:\nAlege unul sau mai multe răspunsuri corecte:\nA: VEȚI CIRCULA CU PRUDENȚĂ PÂNĂ LA PRIMA INTERSECȚIE\nB: VEȚI CIRCULA CU MAXIMUM 30 KM/H PE SECTORUL DE DRUM DENIVELAT\nC: VEȚI CIRCULA CU MINIMUM 30 KM/H', 'response': 'Răspuns corect: B\n\nExplicație: În situația dată circulați pe un drum cu denivelări semnalizat corespunzător, indicatorul de avertizare cu formă triunghiulară se numește “Drum cu denivelări”. La întâlnirea acestui indicator conducătorii auto sunt obligați să reducă viteza de deplasare până la maxim 30 de km/h în localități și 50 de km/h în afara localităților. Prin urmare, fie că sub acest indicator\xa0“Drum cu denivelări” este sau nu instalat indicatorul de limitare de viteză, aveți obligația de a reduce viteza de deplasare\xa0până la ma

In [ ]:
results, acc = evaluate_gemini(
    model,
    dataset=test_ds,
    max_examples=250,
    verbose=True,
    multimodal=True,
    csv_path="gemini2.5_flash_qa_results.csv",
)

In [28]:
selected_indices = [38,51,63,111,199]
filtered_test_ds = test_ds.select(selected_indices)
results, acc = evaluate_gemini(
    model,
    dataset=filtered_test_ds,
    max_examples=250,
    verbose=True,
    multimodal=True,
    csv_path="completion_gemini2.5_flash_qa_results.csv",
)

Evaluare QA:  20%|██        | 1/5 [00:20<01:22, 20.73s/ex]


📘 Prompt: None
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: Ce obligații ai într-o intersecție?
Alege unul sau mai multe răspunsuri corecte:
A: SĂ ACORZI PRIORITATE DE TRECERE ATUNCI CÂND PIETONII ANGAJAȚI REGULAMENTAR ÎN TRAVERSARE SE AFLĂ PE SENSUL TĂU DE MERS
B: SĂ CIRCULI CÂT MAI APROAPE DE BORDURA DIN PARTEA DREAPTĂ A TROTUARULUI
C: SĂ CIRCULI CU O VITEZĂ CARE SĂ NU DEPĂȘEASCĂ 30 KM/H ÎN LOCALITĂȚI SAU 50 KM/H ÎN AFARA ACESTORA, DACĂ DRUMUL ARE CEL MULT O BANDĂ PE SENS, IAR PIETONII AFLAȚI PE TROTUAR ÎN APROPIEREA PĂRȚII CAROSABILE INTENȚIONEAZĂ SĂ TRAVERSEZE
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.
Răspuns:
✅ Gold: {'C', 'A'}
🧠 Pred: {'C', 'A'}
📝 Gemini raw: Răspuns: A, C
Explicație: Ai obligația de a acorda prioritate pietonilor angajați regulamentar în traversare și de a reduce viteza la 30/50 km/h în localități/în afara acestora

Evaluare QA:  40%|████      | 2/5 [00:32<00:46, 15.40s/ex]


📘 Prompt: None
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: Nedeclararea, la cererea poliției, a identității persoanelor cărora deținătorii de vehicule le-au încredințat vehiculele proprii se sancționează cu:
Alege unul sau mai multe răspunsuri corecte:
A: AMENDĂ CONTRAVENȚIONALĂ ȘI PUNCTE DE PENALIZARE
B: AMENDĂ CONTRAVENȚIONALĂ
C: REȚINEREA PERMISULUI DE CONDUCERE
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.
Răspuns:
✅ Gold: {'B'}
🧠 Pred: {'B'}
📝 Gemini raw: Răspuns: B
Nedeclararea, la cererea poliției, a identității persoanelor cărora deținătorii de vehicule le-au încredințat vehiculele se sancționează doar cu amendă contravențională, conform Art. 101 alin. (1) pct. 9 din O.U.G. 195/2002, fără aplicarea de puncte de penalizare sau reținerea permisului de conducere.


Evaluare QA:  60%|██████    | 3/5 [00:40<00:23, 11.89s/ex]


📘 Prompt: None
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: La cererea polițistului rutier, împreună cu reprezentantul autorităților abilitate, conducătorul auto este obligat:
Alege unul sau mai multe răspunsuri corecte:
A: SĂ PREZINTE DOCUMENTELE DE CĂLĂTORIE
B: SĂ-L TRANSPORTE LA UNITATEA DE POLIȚIE
C: SĂ PERMITĂ CONTROLUL STĂRII TEHNICE A VEHICULULUI, PRECUM ȘI AL BUNURILOR TRANSPORTATE
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.
Răspuns:
✅ Gold: {'C'}
🧠 Pred: {'C', 'A'}
📝 Gemini raw: Răspuns: A, C
Conducătorul auto este obligat să prezinte documentele solicitate și să permită controlul tehnic al vehiculului, precum și al bunurilor transportate.


Evaluare QA:  80%|████████  | 4/5 [00:43<00:08,  8.54s/ex]


📘 Prompt: https://huggingface.co/datasets/RKThirteen/ImaginiExamenAuto/resolve/main/Cat_B_poz_561.jpg
Imaginea conține o situație rutieră. Răspunde pe baza acesteia și a întrebării de mai jos.
Întrebare: Intenționați să virați la dreapta. Cum procedați?
Alege unul sau mai multe răspunsuri corecte:
A: TRECEȚI PRIN FAȚA PIETONILOR
B: ACORDAȚI PRIORITATE PIETONILOR
C: ACORDAȚI PRIORITATE NUMAI BICICLIȘTILOR
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.
Răspuns:
✅ Gold: {'B'}
🧠 Pred: {'B'}
📝 Gemini raw: Răspuns: B
Explicație: La virajul la dreapta, trebuie acordată prioritate pietonilor care traversează strada pe care intenționați să virați.


Evaluare QA: 100%|██████████| 5/5 [00:52<00:00, 10.46s/ex]


📘 Prompt: None
Nu este furnizată nicio imagine. Răspunde doar pe baza întrebării de mai jos.
Întrebare: Este interzisă staționarea în dreptul marcajului longitudinal continuu, aplicat între sensurile de circulație?
Alege unul sau mai multe răspunsuri corecte:
A: NU
B: DA, ATUNCI CÂND CONDUCĂTORII CELORLALTE VEHICULE AR FI OBLIGAȚI ASTFEL SĂ TREACĂ PESTE MARCAJUL RESPECTIV
C: LEGAL, NU ESTE PREVĂZUT
Mai întâi oferă doar litera/literele răspunsului corect sub forma:
Răspuns: A
Apoi oferă o explicație succintă pe un rând nou, fără 'Let's think'.
Răspuns:
✅ Gold: {'B'}
🧠 Pred: {'B'}
📝 Gemini raw: Răspuns: B
Explicație: Staționarea este interzisă în dreptul marcajului longitudinal continuu, dacă prin aceasta conducătorii celorlalte vehicule ar fi obligați să treacă peste marcajul respectiv.

📊 Acuratețe totală (exact match): 80.00%
📁 Rezultate salvate în: completion_gemini2.5_flash_qa_results.csv


In [5]:
def create_judging_prompt(row):
    return f"""
Ești un evaluator expert în QA. Îți dau un task QA:

Întrebare: {row['prompt']}
Răspuns corect: {row['golds']}
Răspuns prezis: {row['preds']}
Explicație: {row['response']}

Evalueaza NUMAI calitatea explicației. Este explicația în concordanță cu răspunsul prezis? Dar cu cel corect? Are legătură cu întrebarea 

Judecă în acest format:
- Scor (între 0 și 5): 
- Justificare:
"""

def extract_score(judgment_text):
    match = re.search(r'Score\s*\(?\d\s*to\s*5\)?:?\s*(\d)', judgment_text)
    return int(match.group(1)) if match else None

In [7]:
model_files = glob.glob("*.csv")

for file in model_files:
    df = pd.read_csv(file)
    judgments = []
    scores = []
    if 'gemini_raw' in df.columns:
        df.rename(columns={'gemini_raw': 'response'}, inplace=True)

    for i, row in df.iterrows():
        try:
            prompt = create_judging_prompt(row)
            response = model.generate_content(prompt)
            text = response.text.strip()
            judgments.append(text)
            scores.append(extract_score(text))
            print(f'Completed row {i} for file {file}')
        except Exception as e:
            print(f"Exception occured: {e}")
            
    
    df["judgment"] = judgments
    df["score"] = scores
    df.to_csv(file, index=False)
    print(f"Updated: {file}")


Completed row 0 for file rezultate_llama4_maverick_clean.csv
Completed row 1 for file rezultate_llama4_maverick_clean.csv
Completed row 2 for file rezultate_llama4_maverick_clean.csv
Completed row 3 for file rezultate_llama4_maverick_clean.csv
Completed row 4 for file rezultate_llama4_maverick_clean.csv
Completed row 5 for file rezultate_llama4_maverick_clean.csv
Completed row 6 for file rezultate_llama4_maverick_clean.csv
Completed row 7 for file rezultate_llama4_maverick_clean.csv
Completed row 8 for file rezultate_llama4_maverick_clean.csv
Completed row 9 for file rezultate_llama4_maverick_clean.csv
Completed row 10 for file rezultate_llama4_maverick_clean.csv
Completed row 11 for file rezultate_llama4_maverick_clean.csv
Completed row 12 for file rezultate_llama4_maverick_clean.csv
Completed row 13 for file rezultate_llama4_maverick_clean.csv
Completed row 14 for file rezultate_llama4_maverick_clean.csv
Completed row 15 for file rezultate_llama4_maverick_clean.csv
Completed row 16 f